In [ ]:
import kagglehub
import pandas as pd
import altair as alt

path = kagglehub.dataset_download("rohanrao/nifty50-stock-market-data")

files_f = ['HDFCBANK.csv', 'ICICIBANK.csv', 'SBIN.csv', 'KOTAKBANK.csv']
files_t = ['INFY.csv', 'TCS.csv', 'HCLTECH.csv', 'WIPRO.csv']
files_e = ['RELIANCE.csv', 'ONGC.csv', 'NTPC.csv', 'POWERGRID.csv']

dff = pd.DataFrame()
dft = pd.DataFrame()
dfe = pd.DataFrame()

for x in files_f:
    df_f = pd.read_csv(f"{path}/{x}")
    dff = pd.concat([dff, df_f], ignore_index=True)

for x in files_t:
    df_t = pd.read_csv(f"{path}/{x}")
    dft = pd.concat([dft, df_t], ignore_index=True)

for x in files_e:
    df_e = pd.read_csv(f"{path}/{x}")
    dfe = pd.concat([dfe, df_e], ignore_index=True)

dff['Sector'] = 'Financial'
dft['Sector'] = 'Technology'
dfe['Sector'] = 'Energy'

final = pd.concat([dff, dft, dfe], ignore_index=True)
final['Date'] = pd.to_datetime(final['Date'])

final = final.sort_values(['Sector', 'Symbol', 'Date'])
final['Daily_R'] = final.groupby(['Sector', 'Symbol'])['Close'].pct_change()

df_sector_returns = (final.groupby(['Date', 'Sector'])['Daily_R'].mean().reset_index())

df_sector_returns = df_sector_returns.sort_values(['Sector', 'Date'])

df_sector_returns['Cumulative Return'] = ((1 + df_sector_returns['Daily_R'].fillna(0)).groupby(df_sector_returns['Sector']).cumprod() - 1)

df_plot = (df_sector_returns.set_index('Date').groupby('Sector')['Cumulative Return'].resample('ME').last().reset_index())

line_chart = alt.Chart(df_plot).mark_line().encode(
    x='Date:T',
    y=alt.Y('Cumulative Return:Q', axis=alt.Axis(format='.0%')),
    color=alt.Color('Sector:N', title='Sector'),
    tooltip=[
        'Date:T',
        alt.Tooltip('Cumulative Return:Q', format='.2%'),
        'Sector:N'
    ]
).properties(
    title="Cumulative Returns Over Time by Sector",
    width=800
)

line_chart
line_chart.save("cumulative_returns.html")










📦 EXPORTING TECHNOLOGY SECTOR VISUALIZATIONS


NameError: name 'tech_data' is not defined

In [14]:
final['Year'] = final['Date'].dt.year

df_yearly = final.groupby(['Year', 'Sector']).agg(
    AvgReturn=('Daily_R', 'mean'),
    AvgVolume=('Volume', 'mean')
).reset_index()

df_yearly.to_csv("sector_yearly_data.csv", index=False)

df_yearly.head()


,Year,Sector,AvgReturn,AvgVolume
0,2000,Energy,-0.000135,3.502798e+06
1,2000,Financial,0.000896,6.632694e+05
2,2000,Technology,-0.001139,3.362829e+05
3,2001,Energy,0.000381,1.964672e+06
4,2001,Financial,-0.000796,3.668318e+05
